In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# keras image classification from scratch: (See README for URL)
image_size = (350,230)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "mel-grams",
    label_mode="categorical",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "mel-grams",
    label_mode="categorical",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 999 files belonging to 10 classes.
Using 800 files for training.
Found 999 files belonging to 10 classes.
Using 199 files for validation.


AttributeError: 'BatchDataset' object has no attribute 'classes'

In [3]:
model = keras.Sequential()
# input layer
model.add(keras.Input(shape=(350, 230, 3)))


model.add(layers.experimental.preprocessing.Rescaling(1./255))
model.add(layers.Conv2D(filters=32, kernel_size = (2,3), strides=1, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2,3)))
model.add(layers.Conv2D(filters=64, kernel_size=(2,3), strides=1, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2,3)))
model.add(layers.Conv2D(filters=128, kernel_size=(2,3), strides=1, activation="relu"))
model.add(layers.MaxPooling2D(pool_size=(2,3)))
model.add(layers.Flatten())
# a softmax activation will give us the percent likelyhood that a given image is a certain genre
model.add(layers.Dense(10, activation='softmax'))
model.summary()

keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 350, 230, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 349, 228, 32)      608       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 174, 76, 32)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 173, 74, 64)       12352     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 86, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 22, 128)       49280     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 42, 7, 128)        0

In [4]:
epochs = 50

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)


Epoch 1/50
25/25 [==============================] - 17s 634ms/step - loss: 2.2918 - accuracy: 0.1566 - val_loss: 1.9529 - val_accuracy: 0.2513
Epoch 2/50
25/25 [==============================] - 16s 601ms/step - loss: 1.8384 - accuracy: 0.3275 - val_loss: 1.6503 - val_accuracy: 0.3970
Epoch 3/50
25/25 [==============================] - 15s 593ms/step - loss: 1.4792 - accuracy: 0.4609 - val_loss: 1.3862 - val_accuracy: 0.4573
Epoch 4/50
25/25 [==============================] - 15s 593ms/step - loss: 1.2514 - accuracy: 0.5709 - val_loss: 1.2493 - val_accuracy: 0.5980
Epoch 5/50
25/25 [==============================] - 16s 605ms/step - loss: 1.1395 - accuracy: 0.6271 - val_loss: 1.3077 - val_accuracy: 0.5477
Epoch 6/50
25/25 [==============================] - 16s 597ms/step - loss: 1.0257 - accuracy: 0.6486 - val_loss: 1.1956 - val_accuracy: 0.5678
Epoch 7/50
25/25 [==============================] - 16s 606ms/step - loss: 0.9058 - accuracy: 0.6613 - val_loss: 1.2101 - val_accuracy: 0.5829

In [6]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5

In [7]:
# serialize weights to HDF5
model.save_weights("model_final_weights.h5")
print("Saved model to disk")

Saved model to disk
